In [5]:
import sys
from pathlib import Path
parent_dir = Path('.').resolve().parent
sys.path.insert(0, str(parent_dir))

import numpy as np
import pandas as pd
import math
import random
import statistics
import importlib

import math_expressions as mexpr
importlib.reload(mexpr)

import methods
importlib.reload(methods)

<module 'methods' from '/Users/preetkaria/Desktop/RA/time_to_leave/methods.py'>

In [6]:
alpha_range = [2, 3, 4]
beta_range = [1, 2, 4, 6]
h_range = [1/15, 2/15, 10/15]
c_range = [20, 25, 30, 10]
N_range = range(3, 8)
n_range = [5]

In [7]:
columns = ['alpha', 'beta', 'N', 'n', 'h', 'c', 'mean_n', 'std_n', 'alpha_hat', 'beta_hat', 'u_star', 'u_star_hat', 'z', 'u', 'optimal_cost', 'actual_cost', 'intervals_str']
df = pd.DataFrame(columns=columns)

In [8]:
for _ in range(200):
    alpha = random.choice(alpha_range)
    beta = random.choice(beta_range)
    h = random.choice(h_range)
    c = random.choice(c_range)
    N = random.choice(N_range)
    n = random.choice(n_range)
    intervals = np.random.gamma(shape=alpha, scale=beta, size=N+n)

    mean_n = statistics.mean(intervals[:n])
    std_n = statistics.stdev(intervals[:n])

    alpha_hat, beta_hat = methods.gamma_estimate_parameters(n, intervals)
    alpha_hat = round(alpha_hat) # fractional alpha has no solution for u*, hencing rounding it.

    u = methods.cal_actual_time(n, intervals)
    u_star = methods.get_u_star_binary(N, alpha, beta, h, c)
    u_star_hat = methods.get_u_star_binary(N, alpha_hat, beta_hat, h, c)
    z=u_star / u_star_hat

    optimal_cost = methods.cal_cost(c, h, u, u_star)
    actual_cost = methods.cal_cost(c, h, u, u_star_hat)

    intervals_str = '_'.join([str(x) for x in intervals])
    dic = {'alpha': alpha,
           'beta': beta,
           'N': N,
           'n':n,
           'h': h,
           'c': c,
           'mean_n': mean_n,
           'std_n': std_n,
           'alpha_hat': alpha_hat,
           'beta_hat': beta_hat,
           'u_star': u_star,
           'u_star_hat': u_star_hat,
           'z': z,
           'u': u,
           'optimal_cost': optimal_cost,
           'actual_cost': actual_cost,
           'intervals_str': intervals_str}
    print(dic)
    df.loc[len(df)] = dic


{'alpha': 3, 'beta': 2, 'N': 7, 'n': 5, 'h': 0.13333333333333333, 'c': 25, 'mean_n': 5.66502790882746, 'std_n': 2.0647386933702925, 'alpha_hat': 8, 'beta_hat': 0.7525374879896829, 'u_star': 24.25501108, 'u_star_hat': 29.9272394, 'z': 0.8104660358348988, 'u': 44.34732913814003, 'optimal_cost': 2.678975741085338, 'actual_cost': 1.9226786317520042, 'intervals_str': '6.776855943703113_5.896327621926355_7.885096539276001_5.381100145924954_2.3857592933068767_2.0346046840581193_4.460138952995058_9.170382858606336_5.18529874440349_7.828470329638824_8.331943236437999_7.336490332000205'}
{'alpha': 3, 'beta': 1, 'N': 6, 'n': 5, 'h': 0.13333333333333333, 'c': 30, 'mean_n': 2.22586711720418, 'std_n': 1.5923493804322824, 'alpha_hat': 2, 'beta_hat': 1.1391410249808203, 'u_star': 8.70905977, 'u_star_hat': 5.26215243, 'z': 1.6550375318565218, 'u': 11.859667158112787, 'optimal_cost': 0.42008098508170494, 'actual_cost': 0.8796686304150382, 'intervals_str': '3.036104877300124_4.597785905370493_0.976954395

In [9]:
display(df.head())
methods.plot_plotly(data=df['actual_cost'], data_label='Cost')

,alpha,beta,N,n,h,c,mean_n,std_n,alpha_hat,beta_hat,u_star,u_star_hat,z,u,optimal_cost,actual_cost,intervals_str
0,3,2,7,5,0.133333,25,5.665028,2.064739,8,0.752537,24.255011,29.927239,0.810466,44.347329,2.678976,1.922679,6.776855943703113_5.896327621926355_7.88509653...
1,3,1,6,5,0.133333,30,2.225867,1.592349,2,1.139141,8.709060,5.262152,1.655038,11.859667,0.420081,0.879669,3.036104877300124_4.597785905370493_0.97695439...
2,2,6,6,5,0.066667,10,9.962765,5.213371,4,2.728082,40.363267,42.465320,0.950500,69.952584,1.972621,1.832484,7.879913037393136_12.27975842250213_7.11487803...
3,3,4,6,5,0.133333,30,10.986398,2.007826,30,0.366941,41.873539,54.535429,0.767823,64.682341,3.041174,1.352922,11.274739173848165_10.896847218794615_8.100273...
4,3,6,5,5,0.133333,25,20.661498,15.853897,2,12.164948,53.453765,70.018319,0.763425,54.969448,0.202091,25.000000,45.72400212713191_22.872524896454962_3.5876640...


In [10]:
df.to_csv('../data/gen_data.csv', index=False)